In [1]:
# CONFIGURATION
import json
import pymysql
import pandas as pd

credentials = json.load(open('credentials.json','r'))
connection = pymysql.connect(host=credentials['host'], user=credentials['user'], password=credentials['password'], database=credentials['database'])
cursor=connection.cursor()            

In [ ]:
# CREATE BUSINESS TABLE
cursor.execute('''CREATE TABLE IF NOT EXISTS business  
            (id VARCHAR(32) PRIMARY KEY 
            ,name VARCHAR(255)
            ,address VARCHAR(255)
            ,city VARCHAR(255)
            ,state VARCHAR(32)
            ,stars FLOAT
            ,review_count INT
            ,reservations BOOL DEFAULT NULL
            ,take_out BOOL DEFAULT NULL
            ,delivery BOOL DEFAULT NULL
            ,price_range VARCHAR(32) DEFAULT NULL);''')

In [ ]:
# INSERT BUSINESS TABLE

# Canadian Provinces
# prov = ['AB','BC','MB','NB','NL','NT','NS','NU','ON','PE','QC','SK','YT','NF']

for line in open('business.json', 'r'):
    business = json.loads(line)
    if business['attributes']:
        args = (business['business_id'],business['name'],business['address'],business['city'],business['state'],business['stars'],business['review_count'],(business['attributes'].get('RestaurantsReservations','False'))== 'True',(business['attributes'].get('RestaurantsTakeOut','False'))=='True',(business['attributes'].get('RestaurantsDelivery','False'))=='True',business['attributes'].get('RestaurantsPriceRange2',None))
    else:
        args = (business['business_id'],business['name'],business['address'],business['city'],business['state'],business['stars'],business['review_count'],None,None,None,None)

    cursor.execute('''INSERT INTO business (id, name, address, city, state, stars, review_count, reservations, take_out, delivery, price_range) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);''', args)

connection.commit() 

In [ ]:
# CREATE REVIEW TABLE
cursor.execute('''CREATE TABLE IF NOT EXISTS review  
            (id VARCHAR(32) PRIMARY KEY 
            ,user_id VARCHAR(32)
            ,business_id VARCHAR(32)
            ,stars FLOAT
            ,date DATETIME
            ,text TEXT
            ,useful INT
            ,funny INT
            ,cool INT);''') 

In [ ]:
# INSERT REVIEW TABLE
for line in open('review.json', 'r'):
    review = json.loads(line)
    args = (review['review_id'],review['user_id'],review['business_id'],review['stars'],review['date'],review['text'],review['useful'],review['funny'],review['cool'])
    cursor.execute('''INSERT INTO review (id, user_id, business_id, stars, date, text, useful, funny, cool) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);''', args)

connection.commit()

In [ ]:
# CREATE USER TABLE
cursor.execute('''CREATE TABLE IF NOT EXISTS user  
            (id VARCHAR(32) PRIMARY KEY 
            ,name VARCHAR(32)
            ,review_count INT
            ,yelping_since DATETIME
            ,useful INT
            ,funny INT
            ,cool INT
            ,fans INT
            ,average_stars FLOAT
            );''') 

In [ ]:
# INSERT USER TABLE
for line in open('user.json', 'r'):
    user = json.loads(line)
    args = (user['user_id'],user['name'],user['review_count'],user['yelping_since'],user['useful'],user['funny'],user['cool'],user['fans'],user['average_stars'])
    cursor.execute('''INSERT INTO user (id, name, review_count, yelping_since, useful, funny, cool, fans, average_stars) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);''', args)

connection.commit()

# SECTION 1 - ANALYSIS

### Q2: Top 10 restaurants in Toronto with the highest popularity. You are free to define your ‘popularity’, as long as it can convince people

In [2]:
query = "SELECT * FROM business WHERE STATE = 'ON'"
df_ON = pd.read_sql(query, connection)
df_ON.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33412 entries, 0 to 33411
Data columns (total 11 columns):
id              33412 non-null object
name            33412 non-null object
address         33412 non-null object
city            33412 non-null object
state           33412 non-null object
stars           33412 non-null float64
review_count    33412 non-null int64
reservations    27874 non-null float64
take_out        27874 non-null float64
delivery        27874 non-null float64
price_range     23196 non-null object
dtypes: float64(4), int64(1), object(6)
memory usage: 2.8+ MB


In [3]:
city = df_ON['city'].unique()
## Display all city in ON
city

array(['Etobicoke', 'Toronto', 'Mississauga', 'Richmond Hill',
       'Thornhill', 'Whitby', 'Scarborough', 'Markham', 'Stouffville',
       'Vaughan', 'Brampton', 'Newmarket', 'North York', 'Ajax',
       'Pickering', 'Brooklin', 'Caledon', 'East Mississauga',
       'Whitchurch-Stouffville', 'Woodbridge', 'York', 'Missisauga',
       'Bolton', 'Aurora', 'Oakville', 'Bradford West Gwillimbury',
       'Georgetown', 'East Ajax', 'East York', 'RIchmond Hill',
       'Downtown Toronto', 'Unionville', 'East Gwillimbury', 'King City',
       'Maple', 'Milton', 'Caledon East', 'Concord', 'Gormley',
       'Bradford', 'Toronto-Etobicoke', 'Ashburn', 'Holland Landing',
       'Halton Hills', 'Kleinburg', 'Etobicoke,', 'Beeton', 'NorthYork',
       'Cooksville', 'Rexdale', 'King', 'Mississaugua', 'Uxbridge',
       'Etobiicoke', 'Willowdale', 'Mount Albert', 'Mississagua',
       'Schomberg', 'toronto', 'East Hawkesbury', 'Richmonhill', 'Zephyr',
       'Nobleton', 'Toronto-West', 'Clarkson',


In [4]:
# Filter for restaurants in TORONTO
df_TOR = df_ON.loc[df_ON['city'].isin(['Etobicoke', 'Toronto', 'Mississauga', 'Richmond Hill','Thornhill', 'Scarborough', 'Markham',
       'Vaughan', 'North York','East Mississauga','York', 'Missisauga', 'East York', 'RIchmond Hill',
       'Downtown Toronto', 'Unionville','Toronto-Etobicoke', 
       'Etobicoke,', 'NorthYork','Mississaugua', 'Etobiicoke', 'Willowdale', 'Mississagua','toronto', 'Richmonhill','Toronto-West',  'Yorkdale', 'TORONTO',
      'Richmond Hil','Yorkville',  'Mississauaga',
       'Mississauge', 'North york', 'Centre Island',
       'Scarbrough',  'Don Mills',
       'Vaughn','North  York', 'Tornto', 'Scarobrough',
        'Markham, ON', 'SCARBOROUGH',
       'York Regional Municipality', 'Regional Municipality of York',
       'Thornhil', 'Downsview', 'AGINCOURT', 'markham',
       'Toronto Division', 'THORNHILL', 'ETOBICOKE',
       'Etibicoke', 'Vaughan Mills', 'Northyork', 
        'North Toronto',  'NORTH YORK',
       'West Toronto', 'Agincourt', 'Mississuaga', 'Toronto Scarborough',
       'etobicoke', 'Toronto-North York', 'Woodbridge (Vaughan)'])]

df_TOR.head(5)

,id,name,address,city,state,stars,review_count,reservations,take_out,delivery,price_range
0,__47_7H-yK3HChO5vyut_Q,Instant Muffler and Autorepair,395 Rexdale Boulevard,Etobicoke,ON,1.0,3,NaN,NaN,NaN,None
1,__8j8yhsmE98wNWHJNyAgw,Urawa Sushi,254 Adelaide Street W,Toronto,ON,3.0,86,1.0,1.0,1.0,2
2,__bqGGnOjtY9eEhrZAUsgA,Galangal Thai Fusion,51 Colborne Street,Toronto,ON,3.0,20,0.0,1.0,1.0,2
3,__CQ2SE4NXFFjYfrB_TJ6w,St. Gabriel Medical Clinic,650 Sheppard Avenue E,Toronto,ON,3.0,6,0.0,0.0,0.0,None
4,__knKZ9CUX3L9JBPVwAXEg,Lunch Bag,2601 Matheson Boulevard E,Mississauga,ON,3.0,3,NaN,NaN,NaN,None


In [5]:
# Popularity is defined as mostly reviewed
# Top 10 most popular restaurant in Toronto

df_TOR.sort_values('review_count',ascending=False).head(10)

,id,name,address,city,state,stars,review_count,reservations,take_out,delivery,price_range
24032,r_BrIgzYcwo1NAuG9dLbpg,Pai Northern Thai Kitchen,18 Duncan Street,Toronto,ON,4.5,2121,1.0,1.0,1.0,2
6801,aLcFhMe6DDJ430zelCpd2A,Khao San Road,11 Charlotte Street,Toronto,ON,4.0,1410,0.0,1.0,1.0,2
24851,RtUvSWO_UZ8V3Wpj0n077w,KINKA IZAKAYA ORIGINAL,398 Church St,Toronto,ON,4.0,1397,1.0,1.0,0.0,2
15073,iGEvDk6hsizigmXhDKs2Vg,Seven Lives Tacos Y Mariscos,69 Kensington Avenue,Toronto,ON,4.5,1152,0.0,1.0,0.0,1
20040,N93EYZy9R0sdlEvubu94ig,Banh Mi Boys,392 Queen Street W,Toronto,ON,4.5,1045,0.0,1.0,0.0,1
24929,RwRNR4z3kY-4OsFqigY5sw,Uncle Tetsu's Japanese Cheesecake,598 Bay Street,Toronto,ON,3.5,939,0.0,1.0,0.0,1
32756,zgQHtqX0gqMw1nlBZl2VnQ,Momofuku Noodle Bar,"190 University Avenue, Ground Floor",Toronto,ON,3.0,897,1.0,1.0,1.0,2
27334,uAAWlLdsoUf872F1FKiX1A,Bang Bang Ice Cream and Bakery,93a Ossington Avenue,Toronto,ON,4.0,894,0.0,1.0,0.0,2
627,-av1lZI1JDY_RZN2eTMnWg,Salad King Restaurant,340 Yonge Street,Toronto,ON,3.5,876,1.0,1.0,0.0,2
792,-kG0N8sBhBotMbu0KVSPaw,Toronto Pearson International Airport,3111 Convair Drive,Mississauga,ON,3.0,862,0.0,0.0,0.0,3


### Q3: How many Canadian residents(figure out who are Canadian residents by yourself) reviewed the business “Mon Ami Gabi” in last 1 year?

In [6]:
query = "SELECT * FROM business WHERE name = 'Mon Ami Gabi'"
df = pd.read_sql(query, connection)
df

,id,name,address,city,state,stars,review_count,reservations,take_out,delivery,price_range
0,4JNXUYY8wbaaDmk3BPzlWw,Mon Ami Gabi,3655 Las Vegas Blvd S,Las Vegas,NV,4.0,8348,1,1,0,2


In [7]:
business_id = df.loc[0,'id']

# Get business ID for Mon Ami Gabi
business_id

'4JNXUYY8wbaaDmk3BPzlWw'

In [8]:
# Get all users who reviewed Mon Ami Gabi in the last 1 year
query = f"SELECT distinct user_id FROM review WHERE business_id = '{business_id}' and date >= DATE_SUB(NOW(),INTERVAL 1 YEAR)"
df = pd.read_sql(query, connection)
# Old dataset might have led to the few numbers of results
df

,user_id
0,51DUYf5OcReixM5_hKDIdQ


In [12]:
user_id = df['user_id']
user_id[0]

'51DUYf5OcReixM5_hKDIdQ'

In [13]:
# Get all reviews from those users
query = f'''SELECT review.user_id as user_id, business.id as business_id, business.name as business_name, business.state as business_state
            FROM review LEFT JOIN business
            ON review.business_id = business.id
            WHERE review.user_id = '{user_id}'
        '''
df_users = pd.read_sql(query, connection)

In [14]:
df_users_CA = df_users[df_users['business_state'].isin(['AB','BC','MB','NB','NL','NT','NS','NU','ON','PE','QC','SK','YT','NF'])]
df_users_CA

,user_id,business_id,business_name,business_state


In [15]:
result = df_users_CA.groupby(['user_id']).size().reset_index(name='counts')

In [16]:
# A user who reviewed more than 10 Canadian restaurants is defined as a Canadian Resident
result = result[result['counts'] > 10]
result

,user_id,counts


#### No Canadian users reviewed Mon Ami Gabi in the last one year. This might have been due to the fact that the dataset is out of date.

### Q4: Top 10 most common words in the reviews of the business “Chipotle Mexican Grill” might be helpful and interesting to the business

In [17]:
query = "SELECT * FROM business WHERE name = 'Chipotle Mexican Grill'"
result = pd.read_sql(query, connection)
# Get business ID 
business_id = tuple(result['id'])

In [18]:
query = f'''SELECT text as review_text
            FROM review 
            WHERE business_id in {business_id}
        '''
df_reviews = pd.read_sql(query, connection)
df_reviews

,review_text
0,Super disappointed! Literally got plain white ...
1,Quality of the food is good but what makes me ...
2,My wife and I stopped in here for lunch again ...
3,Good Food. Super Friendly. Fast Service. I hop...
4,"Pretty bomb! Got a bowl, gave me free tortilla..."
...,...
9585,"Decent, filling food served fast. Last visit t..."
9586,OUT OF MARGARITAS ... This should not happen ...
9587,Great place to stop in for a quick burrito and...
9588,Only rated it one star because zero was not a ...


In [19]:
from collections import Counter
import string
def remove_punctuations(text):
    for punctuation in (string.punctuation):
        text = text.replace(punctuation, '')
    
    # Replace newline character
    text = text.replace('\n', '')
    return text.lower()

In [20]:
df_reviews["review_clean"] = df_reviews['review_text'].apply(remove_punctuations)
df_reviews["review_clean"]

0       super disappointed literally got plain white r...
1       quality of the food is good but what makes me ...
2       my wife and i stopped in here for lunch again ...
3       good food super friendly fast service i hope t...
4       pretty bomb got a bowl gave me free tortillas ...
                              ...                        
9585    decent filling food served fast last visit the...
9586    out of margaritas   this should not happen in ...
9587    great place to stop in for a quick burrito and...
9588    only rated it one star because zero was not a ...
9589    worst chipotle i have ever been to they cant s...
Name: review_clean, Length: 9590, dtype: object

In [21]:
# Top 10 most common words for Chipotle Mexican Grill
Counter(" ".join(df_reviews["review_clean"]).split()).most_common(10)

[('the', 42591),
 ('i', 28687),
 ('and', 28313),
 ('to', 24105),
 ('a', 20349),
 ('is', 13652),
 ('of', 12889),
 ('was', 12085),
 ('it', 11486),
 ('this', 10962)]

### Q5: What’s the percentage of users, who reviewed ​“Mon Ami Gabi”, and also reviewed at least 10 other restaurants located in Ontario?

In [22]:
query = "SELECT * FROM business WHERE name = 'Mon Ami Gabi'"
df = pd.read_sql(query, connection)
df.info()
business_id = df.loc[0,'id']

# Get business ID for Mon Ami Gabi
business_id

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
id              1 non-null object
name            1 non-null object
address         1 non-null object
city            1 non-null object
state           1 non-null object
stars           1 non-null float64
review_count    1 non-null int64
reservations    1 non-null int64
take_out        1 non-null int64
delivery        1 non-null int64
price_range     1 non-null object
dtypes: float64(1), int64(4), object(6)
memory usage: 216.0+ bytes


'4JNXUYY8wbaaDmk3BPzlWw'

In [23]:
# Get all users who reviewed Mon Ami Gabi
query = f"SELECT distinct user_id FROM review WHERE business_id = '{business_id}'"
df = pd.read_sql(query, connection)
user_id = tuple(df['user_id'])

In [24]:
# Get all reviews from those users
query = f'''SELECT review.user_id as user_id, business.id as business_id, business.name as business_name, business.state as business_state
            FROM review LEFT JOIN business
            ON review.business_id = business.id
            WHERE review.user_id in {user_id}
        '''
df_users = pd.read_sql(query, connection)

In [25]:
df_users_ON = df_users[df_users['business_state'] == 'ON']

In [26]:
df_users_ON_group = df_users_ON.groupby('user_id').size().reset_index(name='counts')

In [30]:
# Number of users who reviewed 'Mon Ami Gabi' and at least 10 other restaurants located in ON
result = df_users_ON_group[df_users_ON_group['counts'] >= 10]

In [31]:
# Get total number of users
query = "SELECT count(*) FROM user"
result = pd.read_sql(query, connection)
result

,count(*)
0,1637138


In [32]:
percentage = 83/1637138 * 100
percentage

0.005069823069283102

##### About 0.0051% users who reviewed Mon Ami Gabi also reviewed at least 10 restaurants located in Ontario

### Q6: Please think about 2 more analytics, which provide insights for existing/future ​Business Owners​, to make important decisions regarding new business or business expansion.

#### 1. Does business attributes like having delivery/takeout options, having parking space, having reservations contribute to higher rating?

#### 2. What words in a review are important and associated with positive reviews for that specific type of restaurant. Business owners might want to purchase fake reviews with those words to increase popularity.